## Carga librerias necesarias


In [1]:
import pandas as pd 
import numpy as np
import os
from pandas import concat

## Carga del los dataset correspondientes al año 2022
Se cargan los csv de cada mes en una lista

In [2]:
file_names = os.listdir('../data/anio22/')
file_names

['abr_mo22.csv',
 'ago_mo22.csv',
 'dic_mo22.csv',
 'ene_mo22.csv',
 'feb_mo22.csv',
 'jul_mo22.csv',
 'jun_mo22.csv',
 'mar_mo22.csv',
 'may_mo22.csv',
 'nov_mo22.csv',
 'oct_mo22.csv',
 'sep_mo22.csv']

## Crear único dataset con los los datos del año 2022
Recorrer la lista files_names para crear un único dataset


In [3]:
df_aire = pd.DataFrame()
for file_name in file_names:
    df_current = pd.read_csv('../data/anio22/' + file_name, delimiter=';')
    print(file_name, df_current.shape)
    df_aire = pd.concat([df_aire, df_current], ignore_index=True)
    

abr_mo22.csv (3951, 56)
ago_mo22.csv (3960, 56)
dic_mo22.csv (4092, 56)
ene_mo22.csv (3929, 56)
feb_mo22.csv (3556, 56)
jul_mo22.csv (3968, 56)
jun_mo22.csv (3872, 56)
mar_mo22.csv (3946, 56)
may_mo22.csv (4059, 56)
nov_mo22.csv (3960, 56)
oct_mo22.csv (4092, 56)
sep_mo22.csv (3838, 56)


## Filtrar los datos con el componente NO2

In [4]:
df_aire = df_aire[(df_aire['MAGNITUD'] == 14)]
df_aire.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
360,28,79,8,14,28079008_14_6,2022,4,1,21.84,N,...,44.37,N,32.14,N,22.59,N,35.89,N,37.98,N
361,28,79,8,14,28079008_14_6,2022,4,2,42.95,N,...,44.21,N,36.62,N,24.35,N,30.59,N,35.03,N
362,28,79,8,14,28079008_14_6,2022,4,3,23.94,N,...,58.81,N,79.46,N,66.34,N,71.38,N,98.95,N
363,28,79,8,14,28079008_14_6,2022,4,4,76.08,N,...,58.79,V,54.49,V,54.82,V,56.54,V,69.71,V
364,28,79,8,14,28079008_14_6,2022,4,5,77.14,V,...,59.53,V,62.07,V,68.11,V,63.85,V,71.80,V


## Crear una nueva columna fecha 

In [5]:
df_aire['fecha'] = pd.to_datetime(df_aire[['ANO','MES','DIA']].rename(columns = {'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY'}))

## Extraer las datos de las medidas tomadas

In [6]:
data_o3_columns = [ 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24'] # feature
veri_columns = [ 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']

hora_column = np.arange(1,25) # feature label

## Crear un dataset auxiliar con los datos extraidos  y las nuevas columnas creadas

In [7]:


data_o3 = pd.DataFrame(columns=['hora', 'dato', 'verificado'])
aux = pd.DataFrame(columns=['ano', 'mes', 'dia', 'hora', 'dato', 'verificado', 'estacion']) # aux de próxima fila a ser añadida
for  (h,d,v) in zip(hora_column, data_o3_columns, veri_columns): 
     aux['dato'] = df_aire [ data_o3_columns[h-1] ] 
     aux['verificado'] = df_aire [ veri_columns[h-1] ] 
     aux['hora'] = h-1
     aux[['ano', 'mes', 'dia']] = df_aire[['ANO','MES','DIA']]
     aux['estacion'] = df_aire ['ESTACION']
     data_o3 = pd.concat([data_o3, aux], ignore_index=True)
data_o3 = data_o3.astype({'hora': 'int32'}, copy = False)

data_o3.head()



,hora,dato,verificado,ano,mes,dia,estacion
0,0,21.84,N,2022.0,4.0,1.0,8.0
1,0,42.95,N,2022.0,4.0,2.0,8.0
2,0,23.94,N,2022.0,4.0,3.0,8.0
3,0,76.08,N,2022.0,4.0,4.0,8.0
4,0,77.14,V,2022.0,4.0,5.0,8.0


## Añadir la columna fecha en formato datetime al nuevo dataset 

In [8]:
data_o3['fecha'] = pd.to_datetime(data_o3[['ano','mes','dia','hora']].rename(columns = {'ano': 'year', 'mes': 'month','dia': 'day', 'hora': 'hour'}))
data_o3.head()

,hora,dato,verificado,ano,mes,dia,estacion,fecha
0,0,21.84,N,2022.0,4.0,1.0,8.0,2022-04-01
1,0,42.95,N,2022.0,4.0,2.0,8.0,2022-04-02
2,0,23.94,N,2022.0,4.0,3.0,8.0,2022-04-03
3,0,76.08,N,2022.0,4.0,4.0,8.0,2022-04-04
4,0,77.14,V,2022.0,4.0,5.0,8.0,2022-04-05


## Multi-index data frame con estacion y fecha :O

In [9]:
data_o3.set_index(['estacion', 'fecha'], inplace=True)
data_o3.head()

hora   dato verificado     ano  mes  dia
estacion fecha                                               
8.0      2022-04-01     0  21.84          N  2022.0  4.0  1.0
         2022-04-02     0  42.95          N  2022.0  4.0  2.0
         2022-04-03     0  23.94          N  2022.0  4.0  3.0
         2022-04-04     0  76.08          N  2022.0  4.0  4.0
         2022-04-05     0  77.14          V  2022.0  4.0  5.0

## Ordenar por estación y fecha 

In [10]:
data_o3 = data_o3.sort_values(['estacion', 'fecha'])
data_o3.head()



hora   dato verificado     ano  mes  dia
estacion fecha                                                        
8.0      2022-01-01 00:00:00     0  18.39          V  2022.0  1.0  1.0
         2022-01-01 01:00:00     1   8.18          V  2022.0  1.0  1.0
         2022-01-01 02:00:00     2   7.05          V  2022.0  1.0  1.0
         2022-01-01 03:00:00     3  14.22          V  2022.0  1.0  1.0
         2022-01-01 04:00:00     4  24.66          V  2022.0  1.0  1.0

## Rellenar con NaN las filas perdidas 
 Para cada estación, rellenamos con NaN si hay alguna fila perdida y guardamos el resultado en un nuevo dataframe que ya debería tener todas las filas


In [11]:
estaciones = data_o3.index.get_level_values('estacion').unique()
datetime_index = pd.date_range(start='1/1/2022', end='12/31/2022 23:00:00', freq='H', name='fecha')
data_o3_nuevo = pd.DataFrame()
for estacion in estaciones:
    #data_o3_estacion = data_o3[data_o3['estacion'] == estacion]
    data_o3_estacion = data_o3.loc[estacion]
    print(estacion, len(data_o3_estacion))
    data_o3_estacion = data_o3_estacion.reindex(datetime_index)
    data_o3_estacion.reset_index(inplace=True)
    print(estacion, len(data_o3_estacion))
    data_o3_estacion['estacion'] = estacion
    data_o3_nuevo = pd.concat([data_o3_nuevo, data_o3_estacion],ignore_index=True)
    
print(len(data_o3_nuevo))

8.0 8760
8.0 8760
16.0 6336
16.0 8760
17.0 8760
17.0 8760
18.0 8760
18.0 8760
24.0 8736
24.0 8760
27.0 8760
27.0 8760
35.0 8760
35.0 8760
39.0 8760
39.0 8760
49.0 8760
49.0 8760
54.0 8760
54.0 8760
58.0 8760
58.0 8760
59.0 8760
59.0 8760
60.0 8760
60.0 8760
113880


In [12]:
# Multi-index data_o3 frame con estacion y fecha :O
data_o3_nuevo.set_index(['estacion', 'fecha'], inplace=True)

##  Actualizar  data con el nuevo df:

In [13]:
data_o3 = data_o3_nuevo
data_o3.info()
data_o3

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 113880 entries, (8.0, Timestamp('2022-01-01 00:00:00')) to (60.0, Timestamp('2022-12-31 23:00:00'))
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   hora        111432 non-null  float64
 1   dato        111432 non-null  float64
 2   verificado  111432 non-null  object 
 3   ano         111432 non-null  float64
 4   mes         111432 non-null  float64
 5   dia         111432 non-null  float64
dtypes: float64(5), object(1)
memory usage: 5.9+ MB


hora   dato verificado     ano   mes   dia
estacion fecha                                                          
8.0      2022-01-01 00:00:00   0.0  18.39          V  2022.0   1.0   1.0
         2022-01-01 01:00:00   1.0   8.18          V  2022.0   1.0   1.0
         2022-01-01 02:00:00   2.0   7.05          V  2022.0   1.0   1.0
         2022-01-01 03:00:00   3.0  14.22          V  2022.0   1.0   1.0
         2022-01-01 04:00:00   4.0  24.66          V  2022.0   1.0   1.0
...                            ...    ...        ...     ...   ...   ...
60.0     2022-12-31 19:00:00  19.0   1.66          V  2022.0  12.0  31.0
         2022-12-31 20:00:00  20.0   2.14          V  2022.0  12.0  31.0
         2022-12-31 21:00:00  21.0   2.59          V  2022.0  12.0  31.0
         2022-12-31 22:00:00  22.0   5.30          V  2022.0  12.0  31.0
         2022-12-31 23:00:00  23.0   6.12          V  2022.0  12.0  31.0

[113880 rows x 6 columns]

In [14]:
# Mostramos los datos que se habían perdidos
missing_data_o3 = data_o3[data_o3['dato'].isna()]
print(f'Cantidad de datos no existentes (perdidos): {len(missing_data_o3)}')
missing_data_o3

Cantidad de datos no existentes (perdidos): 2448


hora  dato verificado  ano  mes  dia
estacion fecha                                                    
16.0     2022-06-10 00:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-06-10 01:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-06-10 02:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-06-10 03:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-06-10 04:00:00   NaN   NaN        NaN  NaN  NaN  NaN
...                            ...   ...        ...  ...  ...  ...
24.0     2022-04-17 19:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-04-17 20:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-04-17 21:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-04-17 22:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-04-17 23:00:00   NaN   NaN        NaN  NaN  NaN  NaN

[2448 rows x 6 columns]

In [15]:
# Rellenamos las columnas conciernientes a la fecha que faltan en esos datos perdidos excepto la columna 'dato' que ya trataremos más adelante
dt = missing_data_o3.index.get_level_values('fecha')
data_o3.loc[data_o3['dato'].isna(),'ano'] = dt.year
data_o3.loc[data_o3['dato'].isna(),'mes'] = dt.month
data_o3.loc[data_o3['dato'].isna(),'dia'] = dt.day
data_o3.loc[data_o3['dato'].isna(),'hora'] = dt.hour
data_o3 = data_o3.astype({'hora': 'int32'}, copy = False)

#data_o3.loc[data_o3['dato'].isna(),['ano', 'mes', 'dia', 'hora']] = [dt.year, dt.month, dt.day, dt.hour]
data_o3[data_o3['dato'].isna()]

hora  dato verificado     ano  mes   dia
estacion fecha                                                        
16.0     2022-06-10 00:00:00     0   NaN        NaN  2022.0  6.0  10.0
         2022-06-10 01:00:00     1   NaN        NaN  2022.0  6.0  10.0
         2022-06-10 02:00:00     2   NaN        NaN  2022.0  6.0  10.0
         2022-06-10 03:00:00     3   NaN        NaN  2022.0  6.0  10.0
         2022-06-10 04:00:00     4   NaN        NaN  2022.0  6.0  10.0
...                            ...   ...        ...     ...  ...   ...
24.0     2022-04-17 19:00:00    19   NaN        NaN  2022.0  4.0  17.0
         2022-04-17 20:00:00    20   NaN        NaN  2022.0  4.0  17.0
         2022-04-17 21:00:00    21   NaN        NaN  2022.0  4.0  17.0
         2022-04-17 22:00:00    22   NaN        NaN  2022.0  4.0  17.0
         2022-04-17 23:00:00    23   NaN        NaN  2022.0  4.0  17.0

[2448 rows x 6 columns]

In [16]:


# 6. Por último, marcamos los datos no verificados como NaN
no_verificados = data_o3.loc[data_o3['verificado'] != 'V']
print(f'Cantidad de datos existentes pero no válidos:{len(no_verificados)}')
data_o3.loc[data_o3['verificado'] != 'V', ['dato']] = np.nan
data_o3.loc[data_o3['verificado'] != 'V']



Cantidad de datos existentes pero no válidos:4073


hora  dato verificado     ano   mes   dia
estacion fecha                                                         
8.0      2022-01-09 18:00:00    18   NaN          N  2022.0   1.0   9.0
         2022-01-14 18:00:00    18   NaN          N  2022.0   1.0  14.0
         2022-01-19 18:00:00    18   NaN          N  2022.0   1.0  19.0
         2022-01-24 18:00:00    18   NaN          N  2022.0   1.0  24.0
         2022-01-25 11:00:00    11   NaN          N  2022.0   1.0  25.0
...                            ...   ...        ...     ...   ...   ...
60.0     2022-11-21 13:00:00    13   NaN          N  2022.0  11.0  21.0
         2022-11-30 09:00:00     9   NaN          N  2022.0  11.0  30.0
         2022-11-30 10:00:00    10   NaN          N  2022.0  11.0  30.0
         2022-12-21 12:00:00    12   NaN          N  2022.0  12.0  21.0
         2022-12-21 13:00:00    13   NaN          N  2022.0  12.0  21.0

[4073 rows x 6 columns]

In [17]:
# borramos columna verificado
del data_o3['verificado']
data_o3.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 113880 entries, (8.0, Timestamp('2022-01-01 00:00:00')) to (60.0, Timestamp('2022-12-31 23:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   hora    113880 non-null  int32  
 1   dato    109807 non-null  float64
 2   ano     113880 non-null  float64
 3   mes     113880 non-null  float64
 4   dia     113880 non-null  float64
dtypes: float64(4), int32(1)
memory usage: 4.6 MB


In [18]:


# Primero de todo, deshacemos el multíndice
data_o3.reset_index(inplace=True) # Salvar los datos sin indices de estacion y fecha



In [19]:


# 7. Añadimos columna para el día de la semana
#dates = data_o3.index.get_level_values('fecha').to_pydatetime() # con multiindex
dates = data_o3['fecha'].dt.to_pydatetime() # con column y sin multiindex
days_of_week = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
data_o3['dia_semana'] = np.array([days_of_week[d.weekday()] for d in dates])



In [20]:
data_o3 = data_o3.rename(columns={'dato': 'O3'}, copy = False)

In [21]:


data_o3.info()
data_o3.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113880 entries, 0 to 113879
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   estacion    113880 non-null  float64       
 1   fecha       113880 non-null  datetime64[ns]
 2   hora        113880 non-null  int32         
 3   O3          109807 non-null  float64       
 4   ano         113880 non-null  float64       
 5   mes         113880 non-null  float64       
 6   dia         113880 non-null  float64       
 7   dia_semana  113880 non-null  object        
dtypes: datetime64[ns](1), float64(5), int32(1), object(1)
memory usage: 6.5+ MB


,estacion,fecha,hora,O3,ano,mes,dia,dia_semana
0,8.0,2022-01-01 00:00:00,0,18.39,2022.0,1.0,1.0,Sabado
1,8.0,2022-01-01 01:00:00,1,8.18,2022.0,1.0,1.0,Sabado
2,8.0,2022-01-01 02:00:00,2,7.05,2022.0,1.0,1.0,Sabado
3,8.0,2022-01-01 03:00:00,3,14.22,2022.0,1.0,1.0,Sabado
4,8.0,2022-01-01 04:00:00,4,24.66,2022.0,1.0,1.0,Sabado


In [22]:
data_o3.tail()

,estacion,fecha,hora,O3,ano,mes,dia,dia_semana
113875,60.0,2022-12-31 19:00:00,19,1.66,2022.0,12.0,31.0,Sabado
113876,60.0,2022-12-31 20:00:00,20,2.14,2022.0,12.0,31.0,Sabado
113877,60.0,2022-12-31 21:00:00,21,2.59,2022.0,12.0,31.0,Sabado
113878,60.0,2022-12-31 22:00:00,22,5.30,2022.0,12.0,31.0,Sabado
113879,60.0,2022-12-31 23:00:00,23,6.12,2022.0,12.0,31.0,Sabado


In [23]:
# Serie con solamente los datos de contaminación
datos = data_o3['O3']
datos

0         18.39
1          8.18
2          7.05
3         14.22
4         24.66
          ...  
113875     1.66
113876     2.14
113877     2.59
113878     5.30
113879     6.12
Name: O3, Length: 113880, dtype: float64

In [24]:
#datos.to_csv('data_o3_hist.csv') # Salvado serie solo datos históricos
data_o3.to_csv('../data/aire/data_o3_fecha.csv') # Salvado con los datos de las fechas